### Useful ...

In [1]:
%%bash 
uname -a 
date
${PWD}

g++ --version 
cmake --version 
clang++-9 --version
python3 --version

Linux 7905c004e797 5.10.47-linuxkit #1 SMP Sat Jul 3 21:51:47 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux
Wed 01 Jun 2022 09:50:29 PM UTC


bash: line 3: /home/jovyan/work/PyPhysX/Swig: Is a directory


g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

cmake version 3.16.3

CMake suite maintained and supported by Kitware (kitware.com/cmake).
clang version 9.0.1-12 
Target: x86_64-pc-linux-gnu
Thread model: posix
InstalledDir: /usr/bin
Python 3.9.12


### Setting up...

In [2]:
import re
import requests
import numpy

np_version = re.compile(r'(?P<MAJOR>[0-9]+)\.'
                        '(?P<MINOR>[0-9]+)') \
                        .search(numpy.__version__)
np_version_string = np_version.group()
np_version_info = {key: int(value)
                   for key, value in np_version.groupdict().items()}

np_file_name = 'numpy.i'
np_file_url = 'https://raw.githubusercontent.com/numpy/numpy/maintenance/' + \
              np_version_string + '.x/tools/swig/' + np_file_name
if(np_version_info['MAJOR'] == 1 and np_version_info['MINOR'] < 9):
    np_file_url = np_file_url.replace('tools', 'doc')

chunk_size = 8196
with open(np_file_name, 'wb') as file:
    for chunk in requests.get(np_file_url,
                              stream=True).iter_content(chunk_size):
        file.write(chunk)

In [3]:
%%bash 
CHECKING()
{ 
  echo "[DOCKER] LIBRARIES"
  ls -la /home/jovyan/work/physx/install/linux/PhysXSO/bin/linux.clang/release/
  
  echo "[DOCKER] INCLUDES:"
  ls /home/jovyan/work/physx/install/linux/PhysX/include/PxPhysicsAPI.h
  ls /home/jovyan/work/physx/install/linux/PxShared/include/foundation/Px.h
}

CHECKING

[DOCKER] LIBRARIES
total 13224
drwxr-xr-x 1 jovyan root    4096 May 19 16:41 .
drwxr-xr-x 1 jovyan root    4096 May 19 08:51 ..
-rw-r--r-- 1 jovyan root 3641520 May 19 16:38 libPhysX_64.so
-rw-r--r-- 1 jovyan root  197216 May 19 16:39 libPhysXCharacterKinematic_64.so
-rw-r--r-- 1 jovyan root  271572 May 19 08:49 libPhysXCharacterKinematic_static_64.a
-rw-r--r-- 1 jovyan root 1653344 May 19 16:34 libPhysXCommon_64.so
-rw-r--r-- 1 jovyan root  294248 May 19 16:38 libPhysXCooking_64.so
-rw-r--r-- 1 jovyan root 1826768 May 19 16:41 libPhysXExtensions_64.so
-rw-r--r-- 1 jovyan root 2658990 May 19 08:51 libPhysXExtensions_static_64.a
-rw-r--r-- 1 jovyan root   89536 May 19 16:31 libPhysXFoundation_64.so
-rw-r--r-- 1 jovyan root  333856 May 19 16:31 libPhysXPvdSDK_64.so
-rw-r--r-- 1 jovyan root  512040 May 19 08:41 libPhysXPvdSDK_static_64.a
-rw-r--r-- 1 jovyan root  853352 May 19 16:39 libPhysXVehicle_64.so
-rw-r--r-- 1 jovyan root 1165516 May 19 08:49 libPhysXVehicle_static_64.a
[DOCKER] IN

### Compiling...

In [4]:
%%time
%%bash 

SIMULATION_RUN() 
{
  rm -rf BUILD
  mkdir BUILD
  cd BUILD

  cmake .. \
    -DCMAKE_RULE_MESSAGES=OFF \
    -DPHYSX_ROOT_DIR=/home/jovyan/work/physx/install/
    
  make

  bash ../link.txt >> /dev/null
}

SIMULATION_RUN

-- The CXX compiler identification is GNU 9.4.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python3: /opt/conda/bin/python3.9 (found version "3.9.12") found components: Interpreter Development 
-- Found SWIG: /opt/conda/bin/swig (found version "4.0.2") 
-- Configuring done
-- Generating done
-- Build files have been written to: /home/jovyan/work/PyPhysX/Swig/BUILD
Scanning dependencies of target SWIG_fileName_swig_compilation
[ 20%] Built target SWIG_fileName_swig_compilation
Scanning dependencies of target LibraryName
[ 60%] Built target LibraryName
Scanning dependencies of target SWIG_fileName
[100%] Built target SWIG_fileName
CPU times: user 29.4 ms, sys: 26.2 ms, total: 55.6 ms
Wall time: 29.3 s


## Testing...

In [5]:
%%writefile test.py 
#import os; os.system("clear")
import numpy as np

import sys; 
sys.path.append("./BUILD")

import swig_example

## X.1. 
physx = swig_example.Physx() 
physx.Init()
physx.Evolve()

## X.1. 
Keys = swig_example.vectorc(physx.GetKeysSize()); 
physx.GetKeys(Keys)

narray = physx.GetArraySize()
Fields = {k:np.zeros(narray) for k in Keys}

## X.1. 
nsteps = 3
for istep in range(nsteps):
    physx.Evolve()
    for k in Fields.keys() : 
        physx.GetArray(k,Fields[k])
    print(istep, Fields["px"][0])
      
physx.CleanUp()

Overwriting test.py


In [6]:
%%bash 
TESTING()
{
  SWIG_PATH=/home/jovyan/work/PyPhysX/Swig
  cd $SWIG_PATH

  export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/home/jovyan/work/physx/install/linux/PhysXSO/bin/linux.clang/release/
  export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:${SWIG_PATH}/BUILD

  python test.py
}

TESTING

0 -20.000329971313477
1 -20.000568389892578
2 -20.000822067260742
[Physx] Created
[Physx] Init
[Physx] CleanUp
[Physx] End!!



In [7]:
!date

Wed 01 Jun 2022 09:51:02 PM UTC
